In [1]:
import gymnasium as gym
import os
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt

from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.logger import configure

from evaluation.evalcallback_feedback import CurriculumEvalCallback

from gpt.utils import file_to_string

2024-05-24 11:38:38.961463: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 11:38:45.394030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def load_curriculum_training_log(logger_path):
    training_log = np.load(logger_path + "/evaluations.npz", allow_pickle=True)

    reward_dict = training_log["results_dict"]
    success = training_log["successes"].mean(axis=1)

    averaged_dicts = []

    for row in reward_dict:
        sum_dict = {}
        for col in row:
            for key in col:
                sum_dict[key] = sum_dict.get(key, 0) + col[key]

        avg_dict = {key: value/len(row) for key, value in sum_dict.items()}
        averaged_dicts.append(avg_dict)

    reward_df = pd.DataFrame(averaged_dicts)

    return reward_df, success

In [3]:
def load_training_log(logger_path):
    training_log = np.load(logger_path + "/evaluations.npz", allow_pickle=True)
    
    reward_main = training_log["results"].mean(axis=1)
    try:
        success = training_log["successes"].mean(axis=1)
    except:
        success = None
    
    return reward_main, success

In [4]:
def extract_curriculum(logger_path):
    # extract curriculum and return list of dictionaries with task details
    curriculum_txt = file_to_string(logger_path + "curriculum.md")
    # Split the string into individual task sections
    task_sections = re.split(r'\n\n(?=Task)', curriculum_txt)

    # Function to extract details from each task section
    def extract_task_details(task_section):

        details = {}
        lines = task_section.split('\n')
        for line in lines:
            if line.startswith('Task'):
                details['Task'] = line.split(' ')[1]
            elif line.startswith('Name:'):
                details['Name'] = line.split(': ')[1]
            elif line.startswith('Description:'):
                details['Description'] = line.split(': ')[1]
            elif line.startswith('Reason:'):
                details['Reason'] = ': '.join(line.split(': ')[1:])
        return details

    # Extract details for all tasks
    curriculum_info = [extract_task_details(section) for section in task_sections]
    curriculum_length = len(curriculum_info)
    
    return curriculum_info, curriculum_length

In [5]:
def extract_best_agent(logger_path, curriculum_info, curriculum_length):
    task_list = []
    best_agent_list = []
    for idx in range(curriculum_length):
        curriculum_name = curriculum_info[idx]['Name']
        task_list.append(curriculum_name)
        try:
            decision = file_to_string(logger_path + curriculum_name + '.md')
            decision = decision.split('\n')[0]
            numbers = re.findall(r'\d+', decision)
        except:
            numbers = [0]
        if numbers:
            best_agent_list.append(int(numbers[0]))
        else:
            print(f"No number found in the decision {idx}")
            best_agent_list.append(0)
            
    return task_list, best_agent_list

In [ ]:
def summarize_results(logger_path, curriculum_exp_num, her_exp_num, sac_exp_num, scratch_exp_num):
    window_size = 10
    # Load the training logs for curriculum experiment
    curriculum_success_list = []
    for i in range(curriculum_exp_num):
        print("Loading curriculum experiment: ", i)
        curriculum_logger_path = logger_path + "curriculum_" + str(i) + "/"
        curriculum_info, curriculum_length = extract_curriculum(curriculum_logger_path)
        task_list, best_sample_idx = extract_best_agent(curriculum_logger_path, curriculum_info, curriculum_length)

        reward_main, reward_task, success, task_list = [], [], [], []
        for idx, task in enumerate(task_list):
            path = curriculum_logger_path + task + f"/sample_{best_sample_idx[idx]}"
            
            reward_df, success = load_curriculum_training_log(path)

            print("Reward arguments in Task: ", task["Name"])
            for key in reward_df.keys():
                print(key)

            reward_main.append(reward_df["main"])
            reward_task.append(reward_df["task"])
            success_list.append(success)
            task_length.append(len(reward_df["main"]))

        reward_main = np.concatenate(reward_main, axis=0)
        reward_task = np.concatenate(reward_task, axis=0)
        success = np.concatenate(success_list, axis=0)

        # compute the moving average of success rate
        success_moving_avg = pd.Series(success).rolling(window_size).mean().values
        curriculum_success_list.append(success_moving_avg)

    # Load the training logs for HER experiment
    her_success_list = []
    for i in range(her_exp_num):
        print("Loading HER experiment: ", i)
        her_logger_path = logger_path + "her_" + str(i) + "/"
        reward_main, success = load_training_log(her_logger_path)
        
        # compute the moving average of success rate
        success_moving_avg = pd.Series(success).rolling(window_size).mean().values
        her_success_list.append(success_moving_avg)

    # Load the training logs for SAC experiment
    sac_success_list = []
    for i in range(sac_exp_num):
        print("Loading SAC experiment: ", i)
        sac_logger_path = logger_path + "sac_" + str(i) + "/"
        reward_main, success = load_training_log(sac_logger_path)
        
        # compute the moving average of success rate
        success_moving_avg = pd.Series(success).rolling(window_size).mean().values
        sac_success_list.append(success_moving_avg)

    # Load the training logs for Scratch experiment
    scratch_success_list = []
    for i in range(scratch_exp_num):
        print("Loading Scratch experiment: ", i)
        scratch_logger_path = logger_path + "scratch_" + str(i) + "/"
        reward_main, success = load_training_log(scratch_logger_path)
        
        # compute the moving average of success rate
        success_moving_avg = pd.Series(success).rolling(window_size).mean().values
        scratch_success_list.append(success_moving_avg)

    # Plot the success rate in same figure
    curriculum_success_list = np.array(curriculum_success_list)
    her_success_list = np.array(her_success_list)
    sac_success_list = np.array(sac_success_list)
    scratch_success_list = np.array(scratch_success_list)

    curriculum_success_avg = np.mean(curriculum_success_list, axis=0)
    her_success_avg = np.mean(her_success_list, axis=0)
    sac_success_avg = np.mean(sac_success_list, axis=0)
    scratch_success_avg = np.mean(scratch_success_list, axis=0)

    curriculum_success_std = np.std(curriculum_success_list, axis=0)
    her_success_std = np.std(her_success_list, axis=0)
    sac_success_std = np.std(sac_success_list, axis=0)
    scratch_success_std = np.std(scratch_success_list, axis=0)

    # Assign color to each experiments
    curriculum_color = 'blue'
    her_color = 'green'
    sac_color = 'red'
    scratch_color = 'orange'

    # Plot the success rate
    plt.figure()
    plt.plot(curriculum_success_avg, label='Curriculum', color=curriculum_color)
    plt.fill_between(np.arange(len(curriculum_success_avg)), curriculum_success_avg - curriculum_success_std, curriculum_success_avg + curriculum_success_std, color=curriculum_color, alpha=0.2)
    plt.plot(her_success_avg, label='HER', color=her_color)
    plt.fill_between(np.arange(len(her_success_avg)), her_success_avg - her_success_std, her_success_avg + her_success_std, color=her_color, alpha=0.2)
    plt.plot(sac_success_avg, label='SAC', color=sac_color)
    plt.fill_between(np.arange(len(sac_success_avg)), sac_success_avg - sac_success_std, sac_success_avg + sac_success_std, color=sac_color, alpha=0.2)
    plt.plot(scratch_success_avg, label='Scratch', color=scratch_color)
    plt.fill_between(np.arange(len(scratch_success_avg)), scratch_success_avg - scratch_success_std, scratch_success_avg + scratch_success_std, color=scratch_color, alpha=0.2)
    plt.xlabel('Episodes')
    plt.ylabel('Success Rate')
    plt.legend()
    plt.title('Success Rate')
    plt.show()

    # Save the figure to logger path
    plt.savefig(logger_path + "success_rate.png")


In [ ]:
logger_path = "logs/AntMaze_UMaze/"
curriculum_exp_num = 1
her_exp_num = 1
sac_exp_num = 1
scratch_exp_num = 1